# Lab 3: 8-Puzzle Problem

## Problem Overview

The 8-puzzle problem is a classic sliding puzzle consisting of a 3x3 grid with eight numbered tiles and one empty space. The objective is to move the tiles around by sliding them into the empty space to reach a specific goal configuration, typically in ascending order from 1 to 8 with the empty space in the bottom-right corner:

|1|2|3|
|---|---|---|
|4|5|6|
|7|8|0|

Initially the puzzle is in a randomized configuration (initial state), the challenge is to find the shortest path (or the minium number of actions) to order it in the configuration above (With zero in the bottom-right position). In this case, the solution requires 26 moves. The problem is typically solved using search algorithms such as A*, which relies on a heuristic function to guide the search towards the goal.

For the heuristic, we will use Manhattan Distance, which sums the horizontal and vertical distances of each tile from its goal position. This heuristic is effective because it gives an estimate of the number of moves needed for each tile to reach its goal, considering that tiles can only move horizontally or vertically, not diagonally. This approach is commonly known as the city-block distance.


## Let's Code

### Import Code

In [15]:
import heapq
import numpy as np
import logging
from collections import namedtuple
from random import choice

logging.basicConfig(level=logging.DEBUG)

In [2]:
RANDOMIZE_STEPS = 100_000

PUZZLE_DIM = 3
#PUZZLE_DIM = 4
#PUZZLE_DIM = 5


### Useful Functions

Defining some useful function that will be used in the algorithm 

In [16]:
# Defining the tuple that represents an action
action = namedtuple('Action', ['pos1', 'pos2'])

# Defining the function that return the available actions
def available_actions(state: np.ndarray) -> list['action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

# Defining the function that permit to do an action on the actual state
def do_action(state: np.ndarray, action: 'action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

# Defining the Manhattan distance
def manhattan_distance(state):
    dist = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x][y]
            if value != 0:
                target_x, target_y = divmod(value - 1, PUZZLE_DIM)
                dist += abs(target_x - x) + abs(target_y - y)
    return dist

# Function to print puzzle better
def print_puzzle(state: np.ndarray):
    for row in state:
        print(' '.join(f'{num:2}' for num in row))
    print()

## Solution

### Constant Definition

Defining some costants, such as the puzzle dimension and the goal state we want to obtain.

In [ ]:
PUZZLE_DIM = 3

GOAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
GOAL_TUPLE = tuple(map(tuple, GOAL_STATE))  

logging.info(" Showing the final state we want to obtain\n")
print_puzzle(GOAL_STATE)  


INFO:root: Showing the final state we want to obtain



 1  2  3
 4  5  6
 7  8  0



Generating the initial random state

In [5]:
# Generating the state
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for _ in range(RANDOMIZE_STEPS):
    state = do_action(state, choice(available_actions(state)))

logging.info(" Showing the starting state\n")

print_puzzle(state)

INFO:root: Showing the starting state



 0  7  6
 2  8  3
 4  1  5



### A* algorithm (3x3 Puzzle)

In [ ]:
# Defining the A* function

def a_star(initial_state):
    open_set = []
    visited = set()
    
    # Converte lo stato iniziale in una tupla immutabile per essere usata in un insieme
    initial_tuple = tuple(map(tuple, initial_state))
    heapq.heappush(open_set, (0, initial_tuple, 0, [], manhattan_distance(initial_state)))
    
    while open_set:
        #Pop the node with the minimum cost 
        _, current_tuple, g, path, _ = heapq.heappop(open_set)
        # g: number of action
        # path: list of action
        
        #Verify if we reached the GOAL_STATE
        if current_tuple == GOAL_TUPLE:
            print("Puzzle Solved!")
            print("Final state of the puzzle:")
            final_state = np.array(current_tuple)
            print_puzzle(final_state) 
            # returning the number of actions and the list of did actions to solve the puzzle
            return g, path
        
        #Marking the node as visited
        visited.add(current_tuple)
        
        # Converting the current tuple into an np array to be able to compare it
        current_state = np.array(current_tuple)
        
        for action in available_actions(current_state):
            new_state = do_action(current_state, action)
            new_state_tuple = tuple(map(tuple, new_state))

            if new_state_tuple not in visited:
                new_g = g+1
                new_h = manhattan_distance(new_state)
                new_f = new_g + new_h
                heapq.heappush(open_set, (new_f, new_state_tuple, new_g, path + [action], new_h))
            
    return -1, []                
                

In [7]:
#Running the A*

print("Initial state:")
print_puzzle(state)  

num_action, sequence_ofAction = a_star(state)
print(f"Number of action used: {num_action}")
print("Sequence (path) of the actions used:")
for move in sequence_ofAction:
    print(move)

Initial state:
 0  7  6
 2  8  3
 4  1  5

Puzzle Solved!
Final state of the puzzle:
 1  2  3
 4  5  6
 7  8  0

Number of action used: 26
Sequence (path) of the actions used:
Action(pos1=(0, 0), pos2=(1, 0))
Action(pos1=(1, 0), pos2=(1, 1))
Action(pos1=(1, 1), pos2=(2, 1))
Action(pos1=(2, 1), pos2=(2, 0))
Action(pos1=(2, 0), pos2=(1, 0))
Action(pos1=(1, 0), pos2=(1, 1))
Action(pos1=(1, 1), pos2=(0, 1))
Action(pos1=(0, 1), pos2=(0, 2))
Action(pos1=(0, 2), pos2=(1, 2))
Action(pos1=(1, 2), pos2=(1, 1))
Action(pos1=(1, 1), pos2=(0, 1))
Action(pos1=(0, 1), pos2=(0, 0))
Action(pos1=(0, 0), pos2=(1, 0))
Action(pos1=(1, 0), pos2=(2, 0))
Action(pos1=(2, 0), pos2=(2, 1))
Action(pos1=(2, 1), pos2=(1, 1))
Action(pos1=(1, 1), pos2=(1, 2))
Action(pos1=(1, 2), pos2=(2, 2))
Action(pos1=(2, 2), pos2=(2, 1))
Action(pos1=(2, 1), pos2=(1, 1))
Action(pos1=(1, 1), pos2=(1, 0))
Action(pos1=(1, 0), pos2=(2, 0))
Action(pos1=(2, 0), pos2=(2, 1))
Action(pos1=(2, 1), pos2=(1, 1))
Action(pos1=(1, 1), pos2=(1, 2))